# Load csv file to BigQuery

In [3]:
TABLE_NAME="sample" 

In [46]:
### generate connection
import pandas as pd
query = f"select * from {TABLE_NAME}"
data = pd.read_sql(query, con=conn)
data.to_csv(f"{TABLE_NAME}.csv")

## CREATE TABLE and INSERT by SQL

In [37]:
from google.cloud import bigquery

client = bigquery.Client()
project = "projectID"
dataset = "datastet"

Created table innate-plexus-345505.compustat.sec_mthprc_bq


In [54]:
%%time
def upload_csv(file, table_name):
    table_id = f"{project}.{dataset}.{table_name}"

    job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
    )

    with open(file, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_id, job_config=job_config)

    job.result()  # Waits for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

upload_csv(f"{TABLE_NAME}.csv", TABLE_NAME+"_csv")

Loaded 6667519 rows and 12 columns to innate-plexus-345505.compustat.sec_mthprc_csv
CPU times: user 358 ms, sys: 461 ms, total: 819 ms
Wall time: 36.1 s


In [53]:
%%time
def upload_gcs(uri, table_name):
    table_id = f"{project}.{dataset}.{table_name}"

    job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
    )

    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)  # Make an API request.
    print("Loaded {} rows.".format(destination_table.num_rows))


upload_gcs("gs://compustat-lake-test/sample.csv", TABLE_NAME+"_gcs") # 

Loaded 6667519 rows.
CPU times: user 41.6 ms, sys: 4.49 ms, total: 46.1 ms
Wall time: 45.2 s


## 새로운 데이터를 삽입하고 해당 데이터를 검색할 수 있는지 확인
실제 테이블 row수가 6,667,520 -> 6,667,521로 변화된 것을 확인함

In [82]:
def run_bigquery(table_name, query):
    query_job = client.query(query)
    query_job.result()
    return query_job

In [96]:
insert_sample = "INSERT INTO compustat.{} VALUES (3288012, 28883, '91', '2100-03-31', null, 5800.0, 'USD', null, 24.0, 24.0, 22.4, 5418) "
run_bigquery(TABLE_NAME+"_csv", insert_sample.format("sample_csv"))
run_bigquery(TABLE_NAME+"_gcs", insert_sample.format("sample_gcs"))

QueryJob<project=innate-plexus-345505, location=asia-northeast3, id=9d9fa74b-c8e6-415f-a028-53ad3f7d4b74>

In [97]:
res1=  run_bigquery(TABLE_NAME+"_csv", "SELECT datadate FROM compustat.sample_csv WHERE datadate = '2100-03-31'")
res2=  run_bigquery(TABLE_NAME+"_gcs", "SELECT datadate FROM compustat.sample_gcs WHERE datadate = '2100-03-31'")

In [98]:
print("RESULT from sample_csv : ", [r for r in res1])
print("RESULT from sample_gcs : ", [r for r in res2])

RESULT from sec_mthprc_csv :  [Row((datetime.date(2100, 3, 31),), {'datadate': 0})]
RESULT from sec_mthprc_gcs :  [Row((datetime.date(2100, 3, 31),), {'datadate': 0})]
